# Pegando informações dos filmes online

Para conseguir identificar as categorias, diretores e outras informações dos filmes é necessario a utilização de **API's**, para cruzar informações tornar o banco de dados mais confiável.

Usando o **movies_local.csv**, que foi extraido do [Academy Awards Database](http://awardsdatabase.oscars.org/) usando o notebook [scrappy_oscar](./scrapy_oscar.ipynb), com todos os filmes que ja concorreram em alguma categoria o notebook irá buscar respectivamenta as informações dentro da api do TMDB, IMDB *(usando imdbpie)* e Metascore

### 1 - Modelando os dados para a manipulação 

Setando o index do DataFrame como **movie**, organizando por número de vitórias, nomeando as colunas e convertendo o tipo das indicações dos anos em inteiros 


In [1]:
import pandas as pd
import numpy as np

#import csv from database
movies = pd.read_csv('./database/movies_local.csv')
movies = movies.sort_values('win', ascending=False)
movies = movies.set_index('movie')

#rename all columns
movies.columns = ['indication_year', 'number_indications', 'victories']

In [2]:
#convet year into numbers
movies['indication_year'] = movies['indication_year'].astype(int)

print('Movies tem '+str(len(movies)) +' itens')

Movies tem 4770 itens


### 1.1 - Porcentagem de vitórias

Gerando a coluna *percentage_victories* que calcula a porcentagem de vitórias de cada filme com todas as suas participações no Oscar

In [3]:
def percentage_victories(indications, victories):
    return (victories * 100) / indications
movies['percentage_victories'] = np.vectorize(percentage_victories)(movies['number_indications'], movies['victories'])

### 2 - TMDB API: TMDB_ID PT1

Parte do script reponsavel por mandar request para pesquisar no banco de dados da [Api do TMDB](https://developers.themoviedb.org/3) pelo nome do filme e que tenha sido lançado um ano antes da edição do oscar que concorreu

In [4]:
import requests
import json

#KEY from TMDB API
api_key = 'aa043c2ae33192f63fcc150bb97816b7'


def find_movie_tmdb(mv, yr):
    """ Função para procurar o filme no TMDB"""
    
    url = "https://api.themoviedb.org/3/search/movie?api_key="+api_key+"&language=en-US&query="+mv
    url += "&primary_release_year="+str(yr-1)
    payload = "{}"

    response = requests.request("GET", url, data=payload)
    data = json.loads(response.text)
    if response.status_code == 200:
        if len(data['results']) == 1:
            #se existir apnas um resultado retorna o id
            return  data['results'][0]['id']

        elif data['results'] and data['total_results'] > 1:
            #se existir mais de um resultado

            #array para guadar filmes com nome identico
            exact_name = []

            #loop para identificar os filmes com o nome identico ao procurado
            for mvs in data['results']:
                if mvs['title'] == mv:
                    exact_name.append(mvs)

            #retorna o id se o exact_name for igual 1 se não retorna False
            if len(exact_name) == 1:
                return  exact_name[0]['id']
            elif len(exact_name) > 1:
                return  False

            return  False
   
    return  False

Por se tratar de uma grande quantidade de informações foi necessario dividir os filmes em 159 partes, para que a função fosse executada mais rápida

In [5]:
#tmp_movies cria um array de DataFrames com filmes divididos para executar a função
tmp_movies = []

#loop para dividir dataframe movies
for x in range(1,160):
    tmp_movies.append(movies.iloc[int(len(movies.index.values)/159)*(x-1):int(len(movies.index.values)/159)*x,:].copy())

In [6]:
#loop para executar a função find_movie_tmdb e guardar no tmdb_id
for idx, df in enumerate(tmp_movies):
    print(str(idx)+ ' | '+ (50*'='))
    df['tmdb_id'] =  np.vectorize(find_movie_tmdb)(df.index.values, df['indication_year'])

0 | ==================================================
1 | ==================================================
2 | ==================================================
3 | ==================================================
4 | ==================================================
5 | ==================================================
6 | ==================================================
7 | ==================================================
8 | ==================================================
9 | ==================================================
10 | ==================================================
11 | ==================================================
12 | ==================================================
13 | ==================================================
14 | ==================================================
15 | ==================================================
16 | ==================================================
17 | ==================================================
18

146 | ==================================================
147 | ==================================================
148 | ==================================================
149 | ==================================================
150 | ==================================================
151 | ==================================================
152 | ==================================================
153 | ==================================================
154 | ==================================================
155 | ==================================================
156 | ==================================================
157 | ==================================================
158 | ==================================================


Concatenando dataframe e salvando infomações em **movies_TMDB_PI1.csv**

In [3]:
movies_pt1 = pd.concat(tmp_movies)
movies_pt1.to_csv('./database/movies_TMDB_PI1.csv')

NameError: name 'tmp_movies' is not defined

In [8]:
if len(movies_pt1) == len(movies):
    print("Todos os filmes foram buscados no TMDB!")
    
elif len(movies_pt1) > len(movies):
    print("Erro! Existem mais " +len(movies_pt1) - len(movies)+" filmes procurados que o existente no movies.csv")
else:
    print("Erro! Existem menos " +len(movies) - len(movies_pt1)+" filmes procurados que o existente no movies.csv")

Todos os filmes foram buscados no TMDB!


In [20]:
import pandas as pd

movies_pt1 = pd.read_csv('./database/movies_TMDB_PI1.csv').set_index('movie')

In [21]:
moviesdvs_not_tmdb= movies_pt1.loc[movies_pt1['tmdb_id'] <= 1]

len("Existem "+str(len(moviesdvs_not_tmdb))+" filmes que não foram encontrados pelo TMDB")

55

In [22]:
moviesdvs_not_tmdb.head(55)

,indication_year,number_indications,victories,percentage_victories,tmdb_id
movie,,,,,
The Hurt Locker,2010,9,6,66.666667,0
The Godfather Part II,1975,11,6,54.545455,1
A Place in the Sun,1952,9,6,66.666667,1
All about Eve,1951,14,6,42.857143,1
Chicago,2003,13,6,46.153846,1
La La Land,2017,14,6,42.857143,1
Mrs. Miniver,1943,12,6,50.000000,1
Forrest Gump,1995,13,6,46.153846,1
How Green Was My Valley,1942,10,5,50.000000,1


In [ ]:
# def budg_imdb_rev(mvid):
#     if mvid and mvid != 0:
#         url = "https://api.themoviedb.org/3/movie/"+str(mvid)+"?api_key="+api_key+"&language=en-US"

#         payload = "{}"

#         response = requests.request("GET", url, data=payload)
#         data = json.loads(response.text)
#         genres = []
#         production_countries=[]

#         if response.status_code == 200:
#             if data['genres'] and len(data['genres']) > 0:
#                 genres = [x['name'] for x in data['genres']]
            
#             if data['production_countries'] and len(data['production_countries']) > 0:
#                 production_countries = [x['iso_3166_1'] for x in data['production_countries']]
            
#             print(data['title'])
#             return  np.array([data['imdb_id'], data['budget'], data['revenue'], genres, production_countries], dtype=object)
#         print(False)
#         return  np.array([False, False, False, False, False], dtype=object)
    
#     print(False)
#     return  np.array([False, False, False, False, False], dtype=object)


In [ ]:
# moviesdv=[]
# for x in range(1,159):
#     moviesdv.append(moviesdvs.iloc[int(len(moviesdvs.index.values)/159)*(x-1):int(len(moviesdvs.index.values)/159)*x,:].copy())


In [ ]:
# for idx, df in enumerate(moviesdv):
#     now = datetime.datetime.now()
#     print(str(now.hour)+':'+str(now.minute)+':'+str(now.second))
#     print(str(idx)+' | ===================================================================================')
#     #df['tmdb_id'] =  np.vectorize(find_movie)(df.index.values, df['indication_year'])
#     df[['imdb_id','budget','revenue', 'genres', 'production_countries']] = pd.DataFrame(list(df['tmdb_id'].apply(budg_imdb_rev)), index=df.index)


In [ ]:
# moviesdv = pd.concat(moviesdv)

# moviesdv.head()

# moviesdv.to_csv('./database/movies_TMDB_PI2.csv')

#movies.to_csv('./database/movies_TMDB_PI2.csv')

In [ ]:
#https://developers.themoviedb.org/3/movies/get-movie-details
#https://github.com/richardasaurus/imdb-pie#get_title_credits
#https://market.mashape.com/marcalencc/metacritic#
# movies.head()